In [1]:
from os import remove
from py7zr import SevenZipFile
import pandas as pd
import wget
import numpy as np

## Dicionário de Dados
Admitidos/Desligados: Contratação (1), Demissão(2)

Sexo: Masculino (1), Feminino (2)

Classificação de Setores CNAE 2.0: https://docs.google.com/spreadsheets/d/1bMFKpREb4YlW2ZzW1WVLdMN59ol-iLw5/export?format=xlsx

Códigos IBGE por Estado: https://www.oobj.com.br/bc/article/quais-os-c%C3%B3digos-de-cada-uf-no-brasil-465.html

In [2]:
excel = pd.ExcelFile("data/dicionario_dados.xlsx")
get_dict =  lambda x: pd.read_excel(excel, sheet_name=x)

data_dict = {
    sheet: {row[1]: row[2] for row in 
     get_dict(sheet).itertuples()}
    for sheet in excel.sheet_names[1:]
}

### Grandes grupos

In [3]:
big_group = {
	'Administração Pública, Defesa e Seguridade Social': 'Serviços',
	'Agricultura, Pecuária, Produção Florestal, Pesca e AqÜIcultura': 'Agropecuária',
	'Alojamento e Alimentação': 'Serviços',
	'Artes, Cultura, Esporte e Recreação': 'Serviços',
	'Atividades Administrativas e Serviços Complementares': 'Serviços',
	'Atividades Financeiras, de Seguros e Serviços Relacionados': 'Serviços',
	'Atividades Imobiliárias': 'Serviços',
	'Atividades Profissionais, Científicas e Técnicas': 'Serviços',
	'Comércio, Reparação de Veículos Automotores e Motocicletas': 'Comércio',
	'Construção': 'Construção',
	'Educação': 'Serviços',
	'Eletricidade e Gás': 'Indústria',
	'Indústrias Extrativas': 'Indústria',
	'Indústrias de Transformação': 'Indústria',
	'Informação e Comunicação': 'Serviços',
	'Não identificado': 'Não Identificado',
	'Organismos Internacionais e Outras Instituições Extraterritoriais': 'Serviços',
	'Outras Atividades de Serviços': 'Serviços',
	'Saúde Humana e Serviços Sociais': 'Serviços',
	'Serviços Domésticos': 'Serviços',
	'Transporte, Armazenagem e Correio': 'Serviços',
	'Água, Esgoto, Atividades de Gestão de Resíduos e Descontaminação':
	'Indústria'
}

## Leitura e processamento

In [6]:
url = lambda year, month: f"ftp://ftp.mtps.gov.br/pdet/microdados/NOVO CAGED/{year}/{year}{month:02d}/CAGEDMOV{year}{month:02d}.7z"

In [13]:
#Baixando e extraindo microdados
dfs = []
start_year = 2023
start_month = 1
dates = data["competênciamov"].unique()

for year in range(start_year, 2025):
    for month in range(start_month, 13):
        if f"{year}-{month:02d}-01" in dates:
            continue
        try:
            print(f"{month:02d}/{year}")
            wget.download(url(year, month), 'caged.7z')
            archive = SevenZipFile('caged.7z', mode = 'r')
            print('Microdados baixados com sucesso, passando para leitura')
            for name, fd in archive.read(name for name in archive.getnames() if name.endswith(".txt")).items():
                caged_raw = pd.read_csv(fd, delimiter=";", decimal=",")
                caged_raw = caged_raw.loc[caged_raw["uf"] == 25, :].reset_index(drop=True)
                for col in caged_raw.columns:
                    if col in data_dict:
                        caged_raw[f"{col}_cod"] = caged_raw[col]
                        caged_raw[col] = caged_raw[col].apply(lambda x: data_dict[col][x] 
                                                              if x in data_dict[col] else np.nan)

            dfs.append(caged_raw)
            archive.close()
            remove('caged.7z')
            print('Leitura concluida com sucesso')
        except:
            print('Os microdados do mês selecionado ainda não estão disponíveis')
            break

01/2023
-1 / unknownMicrodados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
02/2023
-1 / unknownMicrodados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
03/2023
-1 / unknownMicrodados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
04/2023
-1 / unknownMicrodados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
05/2023
-1 / unknownMicrodados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
06/2023
-1 / unknownMicrodados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
07/2023
-1 / unknownMicrodados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
08/2023
-1 / unknownMicrodados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
09/2023
-1 / unknownMicrodados baixados com sucesso, passando para leitura
Leitura concluida com sucesso
10/2023
-1 / unknownMicrodados baixados com sucesso, pa

In [14]:
new_data = pd.concat(dfs)

In [16]:
data = pd.concat([new_data, data])

In [30]:
data.to_csv("data/caged_pb.csv")

In [37]:
data["competênciamov"] = pd.to_datetime(data["competênciamov"], format="%Y%m")

In [ ]:
data.groupby(["competênciamov", "saldomovimentação"], 
             as_index=False).count()["competênciamov", "saldomovimentação", "idade"](orient="records")

In [17]:
data = pd.read_csv("data/caged_pb.csv", index_col=0)
data["competênciamov"] = pd.to_datetime(data["competênciamov"])

In [31]:
g = pd.read_csv("data/caged_agregado.csv", index_col=0)
g["competênciamov"] = pd.to_datetime(g["competênciamov"])

In [32]:
g2 = g.astype({"competênciamov": str}).groupby(["competênciamov",
                                                            "saldomovimentação"], as_index=False).sum()
g2["graudeinstrução"] = g2["graudeinstrução"] * g2["saldomovimentação"]

g2 = g2.groupby("competênciamov").sum()

In [33]:
g2["graudeinstrução"] = g2["graudeinstrução"] * g2["saldomovimentação"]

g2 = g2.groupby("competênciamov").sum()

In [38]:
g2 = g.astype({"competênciamov": str}).groupby(["competênciamov",
                                                            "saldomovimentação"], as_index=False).sum()
g2["graudeinstrução"] = g2["graudeinstrução"] * g2["saldomovimentação"]
g2.groupby("competênciamov").sum()["graudeinstrução"]

In [43]:
g2.groupby("competênciamov").sum()["graudeinstrução"].to_dict()

{'2023-01-01': -1723,
 '2023-02-01': 514,
 '2023-03-01': -815,
 '2023-04-01': -3181,
 '2023-05-01': 2868,
 '2023-06-01': -223,
 '2023-07-01': 3477,
 '2023-08-01': 8782,
 '2023-09-01': 4193,
 '2023-10-01': 3773,
 '2023-11-01': 3657,
 '2023-12-01': -1504,
 '2024-01-01': 332,
 '2024-02-01': -9}

In [ ]:
a = data.groupby(["competênciamov", "saldomovimentação"]).count()["região"].to_dict()

In [43]:
saldo = data.groupby("competênciamov").sum()["saldomovimentação"]

In [47]:
saldo.values

array([332,  -9])

In [25]:
data = pd.read_csv("data/caged_pb.csv", index_col=0, low_memory=False)
data["competênciamov"] = data["competênciamov"].apply(lambda x: f"{x[:4]}-{x[4:]}-01" if "-" not in x else x)

g = data.groupby(["competênciamov", "município", "seção", "saldomovimentação", "sexo"], as_index=False).agg({
    "graudeinstrução": "count",
})
g.to_csv("data/caged_agregado.csv")

In [21]:
def get_admissions_demissions(data, year, month, category=None):
    data["competênciamov"] = pd.to_datetime(data["competênciamov"])

TypeError: 'dict_keys' object is not subscriptable

In [37]:
data = pd.read_csv("data/caged_agregado.csv", index_col=0)
data["competênciamov"] = pd.to_datetime(data["competênciamov"])

In [44]:
g = data.groupby(["competênciamov", "saldomovimentação"], as_index=False).sum()
g["admissao"] = g["saldomovimentação"] * g["graudeinstrução"]
g["demissao"] = g["saldomovimentação"] * g["graudeinstrução"] * -1

for col in ["admissao", "demissao"]:
    g[col] = g[col].apply(lambda x: x if x > 0 else np.nan)
    
g["admissao"].fillna(method="bfill", inplace=True)
g["demissao"].fillna(method="ffill", inplace=True)

g = g[["competênciamov", "demissao", "admissao"]].drop_duplicates(keep="first").reset_index(drop=True)
g = g.astype({"demissao": int, "admissao": int})

In [55]:
year = 2024
month = 3
g[g["competênciamov"] == f"{year}-{month:02d}-01"].astype({"competênciamov": str}).to_dict(orient="records")

[]

In [60]:
g.set_index("competênciamov")

ModuleNotFoundError: No module named 'flask'

array(['2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01',
       '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01',
       '2023-09-01', '2023-10-01', '2023-11-01', '2023-12-01',
       '2024-01-01', '2024-02-01'], dtype=object)